In [2]:
import os
import pandas as pd

In [3]:
file_path = "../Data_Raw/Tickets_Data"
tables = {}

In [4]:
# Load all Excel files in the directory
for file in os.listdir(file_path):
    if file.endswith(".xlsx"):
        file_path_full = os.path.join(file_path, file)
        tables[file] = pd.read_excel(file_path_full)

In [5]:
adjusted_tables = tables
adjusted_tables["2022.xlsx"].head()

,Datum,Bezoekers,Omzet Elli,Gem. Omzet,Artikel,Bezoekersgroep
0,2022-01-26 10:02:01.930000,1,19.724771,19.724771,NEMO e-ticket age 4-12 GT,Recreatief Buitenland
1,2022-01-26 10:02:03.353000,1,19.724771,19.724771,NEMO e-ticket age 4-12 GT,Recreatief Buitenland
2,2022-01-26 10:02:03.853000,1,19.724771,19.724771,NEMO e-ticket age 4-12 GT,Recreatief Buitenland
3,2022-01-26 10:02:05.010000,1,19.724771,19.724771,NEMO e-ticket adult GT,Recreatief Buitenland
4,2022-01-26 10:02:32.803000,1,19.724771,19.724771,NEMO e-ticket 4-12 jaar GT,Recreatief NL


In [6]:
# Clean and engineer features for each table
for file_name, df in adjusted_tables.items():
    df["Datum"] = df["Datum"].replace("-", pd.NA)
    df['Datum'] = pd.to_datetime(df['Datum'])
    df['Date'] = df['Datum'].dt.date
    df['Hour'] = df['Datum'].dt.hour
    df = df[(df['Hour'] >= 10) & (df['Hour'] <= 17)]
    adjusted_tables[file_name] = df

In [7]:
# Aggregate visitors per day and group
visitors_per_day = {}
combined_pivot_table = pd.DataFrame()

for file_name, df in adjusted_tables.items():
    visitors_per_day_group = df.groupby(['Date', 'Bezoekersgroep'])['Bezoekers'].sum().reset_index()
    pivot_table = visitors_per_day_group.pivot(
        index='Date',
        columns='Bezoekersgroep',
        values='Bezoekers'
    ).fillna(0)
    pivot_table = pivot_table.astype(int)
    pivot_table['Total'] = pivot_table.sum(axis=1)
    combined_pivot_table = pd.concat([combined_pivot_table, pivot_table])
    visitors_per_day[file_name] = pivot_table

# Appended Table

In [8]:
combined_pivot_table = combined_pivot_table.sort_values("Date")

In [9]:
cleaned_data_path = "../../Data_Sources/Data_Modelling/Visitors"
os.makedirs(cleaned_data_path, exist_ok=True)
output_file = os.path.join(cleaned_data_path, "entrance_data_cleaned.csv")
combined_pivot_table.to_csv(output_file, index=True)

# Hourly attendance

In [10]:
# Hourly attendance
visitors_per_hour = {}
combined_hourly_table = pd.DataFrame()

for file_name, df in adjusted_tables.items():
    visitors_per_hour_group = df.groupby(['Date', 'Hour'])['Bezoekers'].sum().reset_index()
    pivot_table = visitors_per_hour_group.pivot(
        index='Date',
        columns='Hour',
        values='Bezoekers'
    ).fillna(0)
    pivot_table["Total"] = pivot_table.sum(axis=1)
    pivot_table = pivot_table.astype(int)
    pivot_table = pivot_table.reset_index()
    combined_hourly_table = pd.concat([combined_hourly_table, pivot_table])
    visitors_per_hour[file_name] = pivot_table

In [11]:
combined_hourly_table = combined_hourly_table.sort_values("Date")

In [12]:
combined_hourly_table['Date'] = pd.to_datetime(combined_hourly_table['Date'])
combined_hourly_table['Year'] = combined_hourly_table['Date'].dt.year
combined_hourly_table['Weekday'] = combined_hourly_table['Date'].dt.day_name()

In [13]:
# Calculate hourly averages by year and weekday
all_columns = combined_hourly_table.columns.tolist()
hour_columns = []
for col in all_columns:
    if isinstance(col, (int, float)) or (isinstance(col, str) and col.replace('.', '').isdigit()):
        if (isinstance(col, str) and col.endswith('.0')) or isinstance(col, (int, float)):
            hour_columns.append(col)

agg_dict = {col: 'mean' for col in hour_columns}
if 'Total' in all_columns:
    agg_dict['Total'] = 'mean'

hourly_avg_by_year_weekday = combined_hourly_table.groupby(
    ['Year', 'Weekday']
).agg(agg_dict).reset_index()

numeric_columns = hour_columns.copy()
if 'Total' in all_columns:
    numeric_columns.append('Total')
hourly_avg_by_year_weekday[numeric_columns] = hourly_avg_by_year_weekday[numeric_columns].round(1)

weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
hourly_avg_by_year_weekday['Weekday_order'] = hourly_avg_by_year_weekday['Weekday'].map(
    {day: i for i, day in enumerate(weekday_order)}
)
hourly_avg_by_year_weekday = hourly_avg_by_year_weekday.sort_values(
    ['Year', 'Weekday_order']
).drop('Weekday_order', axis=1)

# Save hourly averages
output_file = os.path.join(cleaned_data_path, "hourly_visitors.csv")
hourly_avg_by_year_weekday.to_csv(output_file, index=False)

# Add all days to the combined entrance

In [14]:
# Fill missing days in entrance data with zeros
df = pd.read_csv("../Data_Modelling/Visitors/entrance_data_cleaned.csv")
df = df.dropna()
df['Date'] = pd.to_datetime(df['Date'])
start_date = df['Date'].min()
end_date = df['Date'].max()
date_range = pd.date_range(start=start_date, end=end_date)
complete_df = pd.DataFrame({'Date': date_range})
filled_df = pd.merge(complete_df, df, on='Date', how='left')
columns_to_fill = [
    'Extern', 'PO', 'Recreatief Buitenland', 'Recreatief NL', 'Student', 'VO', 'Total'
]
filled_df[columns_to_fill] = filled_df[columns_to_fill].fillna(0).astype(int)
filled_df = filled_df.sort_values('Date')
output_file = os.path.join(cleaned_data_path, "entrance_data_cleaned.csv")
filled_df.to_csv(output_file, index=False)